# 导入需要的环境

In [2]:
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

# 生成数据集

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype = torch.float)
labels = true_w[0]* features[:,0] + true_w[1] * features[:,1] + true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()), dtype=torch.float)

# 读取数据

In [4]:
import torch.utils.data as Data

batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [5]:
for X, y in data_iter:
    print(X, y)
    break


tensor([[-1.8072,  2.3855],
        [-0.1833,  1.6525],
        [ 2.0930, -0.1676],
        [-1.6016,  0.7205],
        [ 0.6360,  0.1085],
        [-0.3309,  0.6721],
        [-0.1767,  0.7332],
        [-0.1416, -0.0342],
        [-2.0448, -0.1425],
        [-0.8617,  0.4645]]) tensor([-7.5353, -1.7762,  8.9621, -1.4642,  5.0956,  1.2620,  1.3668,  4.0560,
         0.5904,  0.9009])


# 定义模型

In [6]:
class LinearNet(torch.nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = torch.nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [7]:
for param in net.parameters():
    print(param)


Parameter containing:
tensor([[-0.3132, -0.2298]], requires_grad=True)
Parameter containing:
tensor([-0.4791], requires_grad=True)


# 初始化模型参数

In [8]:
from torch.nn import init

init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)


Parameter containing:
tensor([0.], requires_grad=True)

# 定义损失函数

In [9]:
loss = torch.nn.MSELoss()

# 定义优化算法

In [10]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [11]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000331
epoch 2, loss: 0.000069
epoch 3, loss: 0.000043


In [16]:
dense = net.linear
print(true_w, dense.weight)
print(true_b, dense.bias)


[2, -3.4] Parameter containing:
tensor([[ 1.9998, -3.3999]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2014], requires_grad=True)
